In [21]:
import numpy as np
import pandas as pd
import os, time, warnings, shap, optuna, random
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.svm import SVC
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, precision_recall_curve, auc
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier

pd.set_option('display.max_columns', 20)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.expand_frame_repr', False)
warnings.filterwarnings('ignore')


In [22]:
df = pd.read_csv('../input/tabular-playground-series-aug-2022/train.csv')
display(df.shape, df.head(), df.failure.value_counts(), df.count())
df0 = df.copy()
df.drop(columns = ['id'], inplace=True)

(26570, 26)

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


0    20921
1     5649
Name: failure, dtype: int64

id                26570
product_code      26570
loading           26320
attribute_0       26570
attribute_1       26570
attribute_2       26570
attribute_3       26570
measurement_0     26570
measurement_1     26570
measurement_2     26570
measurement_3     26189
measurement_4     26032
measurement_5     25894
measurement_6     25774
measurement_7     25633
measurement_8     25522
measurement_9     25343
measurement_10    25270
measurement_11    25102
measurement_12    24969
measurement_13    24796
measurement_14    24696
measurement_15    24561
measurement_16    24460
measurement_17    24286
failure           26570
dtype: int64

In [23]:
# train-test split:

test_size = 0.1
df.reset_index(inplace=True, drop=True)
test_index = random.sample(list(df.index), int(test_size*df.shape[0]))
train = df.iloc[list(set(df.index)-set(test_index))]
test = df.iloc[test_index]
display(train.shape, test.shape, train.head(3), test.head(3), train.count())

(23913, 26)

(2657, 26)

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0


,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
22410,22410,E,111.90,material_7,material_6,6,9,4,8,4,...,11.016,16.298,19.631,12.903,15.139,19.550,NaN,18.789,486.031,0
8989,8989,B,112.39,material_5,material_5,8,8,2,3,6,...,8.959,18.204,18.776,12.172,17.399,16.483,13.272,15.504,650.540,0
5918,5918,B,109.25,material_5,material_5,8,8,5,7,7,...,9.607,18.414,16.056,12.659,17.568,14.484,13.759,15.393,667.989,1


id                23913
product_code      23913
loading           23689
attribute_0       23913
attribute_1       23913
attribute_2       23913
attribute_3       23913
measurement_0     23913
measurement_1     23913
measurement_2     23913
measurement_3     23570
measurement_4     23433
measurement_5     23315
measurement_6     23189
measurement_7     23067
measurement_8     22958
measurement_9     22822
measurement_10    22735
measurement_11    22570
measurement_12    22462
measurement_13    22310
measurement_14    22227
measurement_15    22121
measurement_16    22014
measurement_17    21869
failure           23913
dtype: int64

In [24]:
num_feat = [col for col in train.columns if train[col].nunique()>=20]
cat_te_feat = [col for col in train.columns if train[col].nunique() in (range(5,20))]
cat_ohe_feat = [col for col in train.columns if train[col].nunique()<5]
cat_ohe_feat.remove('failure')
display('num features: ', num_feat, '/n',
       'cat features: ', cat_te_feat+cat_ohe_feat)

'num features: '

['id',
 'loading',
 'measurement_0',
 'measurement_1',
 'measurement_2',
 'measurement_3',
 'measurement_4',
 'measurement_5',
 'measurement_6',
 'measurement_7',
 'measurement_8',
 'measurement_9',
 'measurement_10',
 'measurement_11',
 'measurement_12',
 'measurement_13',
 'measurement_14',
 'measurement_15',
 'measurement_16',
 'measurement_17']

'/n'

'cat features: '

['product_code', 'attribute_0', 'attribute_1', 'attribute_2', 'attribute_3']

In [25]:
# fill na:

for col in num_feat:
    train[col] = train[col].fillna(train[col].median())
    test[col] = test[col].fillna(train[col].median())

for col in cat_te_feat+cat_ohe_feat:
    train[col]=train[col].fillna(train[col].mode()[0])
    test[col]=test[col].fillna(train[col].mode()[0])
    
display(train.count(), test.count())

id                23913
product_code      23913
loading           23913
attribute_0       23913
attribute_1       23913
attribute_2       23913
attribute_3       23913
measurement_0     23913
measurement_1     23913
measurement_2     23913
measurement_3     23913
measurement_4     23913
measurement_5     23913
measurement_6     23913
measurement_7     23913
measurement_8     23913
measurement_9     23913
measurement_10    23913
measurement_11    23913
measurement_12    23913
measurement_13    23913
measurement_14    23913
measurement_15    23913
measurement_16    23913
measurement_17    23913
failure           23913
dtype: int64

id                2657
product_code      2657
loading           2657
attribute_0       2657
attribute_1       2657
attribute_2       2657
attribute_3       2657
measurement_0     2657
measurement_1     2657
measurement_2     2657
measurement_3     2657
measurement_4     2657
measurement_5     2657
measurement_6     2657
measurement_7     2657
measurement_8     2657
measurement_9     2657
measurement_10    2657
measurement_11    2657
measurement_12    2657
measurement_13    2657
measurement_14    2657
measurement_15    2657
measurement_16    2657
measurement_17    2657
failure           2657
dtype: int64

In [26]:
# take out target

X_train = train
y_train = X_train.pop('failure')
X_test = test
y_test = X_test.pop('failure')

display(X_train.shape, X_test.shape, y_train.shape)

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,20.155,10.672,15.859,17.594,15.193,15.029,16.043,13.034,14.6840,764.100
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.6310,682.057
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,18.288,12.715,15.607,19.211,13.798,16.711,18.631,14.094,17.9460,663.376
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,19.060,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.1720,826.282
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,18.093,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.4120,579.885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26564,26564,E,124.77,material_7,material_6,6,9,3,12,7,...,17.191,11.945,15.738,19.723,12.989,16.084,14.616,15.394,16.5100,536.149
26565,26565,E,158.95,material_7,material_6,6,9,6,16,4,...,19.354,11.429,12.177,17.942,10.112,15.795,18.572,16.144,16.4365,729.131
26566,26566,E,146.02,material_7,material_6,6,9,10,12,8,...,19.563,11.242,14.179,20.564,10.234,14.450,14.322,13.146,16.4710,853.924
26568,26568,E,106.38,material_7,material_6,6,9,2,9,4,...,19.358,11.392,17.064,17.814,14.928,16.273,15.485,13.624,12.8650,730.156


In [27]:
# encode cat features, ohe for now:


